# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, get_stops_utils, psth_postprocessing, psth_stats, compare_events, dpca_utils


import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
# pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
#     pn.planning_data_by_point)
#pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

if not hasattr(pn, 'spikes_df'):
    pn.retrieve_or_make_monkey_data()
    pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted,
                                                            sampling_rate=pn.sampling_rate)

# NEXT: try stop end time instead

# Get captures

In [ ]:
reload(get_stops_utils)

In [ ]:
# Example wiring (mirrors your original usage)
valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    capture_match_window=0.3,
    distance_thresh=25.0,
    distance_col="distance_from_ff_to_stop",
)


# Get misses

##  one

In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]

pn.make_one_stop_w_ff_df()
one_stop_miss_df = pn.one_stop_w_ff_df[['first_stop_point_index', 'first_stop_time', 'latest_visible_ff', 'ff_distance', 'min_distance_from_adjacent_stops']].copy()
one_stop_miss_df.rename(columns={'first_stop_point_index': 'stop_point_index', 'first_stop_time': 'stop_time'}, inplace=True)
one_stop_miss_df[columns_to_add] = pn.monkey_information.loc[one_stop_miss_df['stop_point_index'], columns_to_add].values

## more

In [ ]:
pn.get_try_a_few_times_info()
pn.get_give_up_after_trying_info()


In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]
shared_columns = ["stop_point_index", "stop_time"] + columns_to_add

# --- Build expanded + ordered tables for GUAT / TAFT ---
GUAT_expanded = get_stops_utils._expand_trials(pn.GUAT_trials_df, pn.monkey_information)
TAFT_expanded = get_stops_utils._expand_trials(pn.TAFT_trials_df, pn.monkey_information)

# add stop_id to GUAT_trials_df and TAFT_trials_df
GUAT_expanded[columns_to_add] = pn.monkey_information.loc[GUAT_expanded['stop_point_index'], columns_to_add].values
TAFT_expanded[columns_to_add] = pn.monkey_information.loc[TAFT_expanded['stop_point_index'], columns_to_add].values


GUAT = get_stops_utils._add_cluster_ordering(GUAT_expanded)
TAFT = get_stops_utils._add_cluster_ordering(TAFT_expanded)

# --- Per-cluster slices (consistent, vectorized) ---
# First stop in each cluster
GUAT_first = GUAT[GUAT["is_first"]].reset_index(drop=True)
TAFT_first = TAFT[TAFT["is_first"]].reset_index(drop=True)

# Last stop in each cluster
giveup_GUAT_last = GUAT[GUAT["is_last"]].reset_index(drop=True)
capture_TAFT_last = TAFT[TAFT["is_last"]].reset_index(drop=True)

# Middle stops (exclude first and last)
GUAT_middle = GUAT[GUAT["is_middle"]].reset_index(drop=True)
TAFT_middle = TAFT[TAFT["is_middle"]].reset_index(drop=True)

# “First several” = all but the last stop in each cluster
persist_GUAT_nonfinal = GUAT[GUAT["order_in_cluster"] < GUAT["cluster_size"] - 1].reset_index(drop=True)
persist_TAFT_nonfinal = TAFT[TAFT["order_in_cluster"] < TAFT["cluster_size"] - 1].reset_index(drop=True)

# Combine the “first several” from both, keep only columns you care about, then sort by index
both_nonfinal = (
    pd.concat(
        [
            persist_GUAT_nonfinal[shared_columns],
            persist_TAFT_nonfinal[shared_columns],
        ],
        ignore_index=True
    )
    .sort_values("stop_point_index")
    .reset_index(drop=True)
)

persist_both_first = pd.concat([GUAT_first[shared_columns], 
                         TAFT_first[shared_columns]])

both_middle = pd.concat([GUAT_middle[shared_columns], 
                         TAFT_middle[shared_columns]])

# Optional: if you also want “last several” (all but the first), it’s symmetrical:
# giveup_GUAT_last_several = GUAT[GUAT["order_in_cluster"] > 0].reset_index(drop=True)
# capture_TAFT_last_several = TAFT[TAFT["order_in_cluster"] > 0].reset_index(drop=True)

giveup_GUAT_last_plus_single_miss = pd.concat([giveup_GUAT_last[shared_columns], 
                                         one_stop_miss_df[shared_columns]])

all_misses = pd.concat([one_stop_miss_df[shared_columns], 
                                         GUAT_expanded[shared_columns],
                                         persist_TAFT_nonfinal[shared_columns]
                                         ])

all_first_misses = pd.concat(
    [one_stop_miss_df[shared_columns], GUAT_first[shared_columns], TAFT_first[shared_columns]],
    ignore_index=True
)

# captures not in TAFT last (assuming capture_TAFT_last is a subset of captures)
captures_minus_TAFT_last = compare_events.diff_by(valid_captures_df, capture_TAFT_last, key='stop_id')

# non-captures excluding those flagged as 'all_misses'
non_captures_minus_all_misses = compare_events.diff_by(filtered_no_capture_stops_df, all_misses, key='stop_id')


# ===COMPARE EVENTS===

In [ ]:
# ---------- dataset registry (canonical) ----------
datasets_raw = {
    'captures': valid_captures_df.copy(),
    'no_capture': filtered_no_capture_stops_df.copy(),
    'persist_nonfinal': both_nonfinal.copy(),
    'persist_middle': both_middle.copy(),
    'giveup_GUAT_last': giveup_GUAT_last.copy(),
    'capture_TAFT_last': capture_TAFT_last.copy(),
    'giveup_single_miss': one_stop_miss_df.copy(),
    'persist_both_first': persist_both_first.copy(),
    'persist_GUAT_nonfinal': persist_GUAT_nonfinal.copy(),
    'persist_TAFT_nonfinal': persist_TAFT_nonfinal.copy(),
    'giveup_GUAT_last_plus_single_miss': giveup_GUAT_last_plus_single_miss.copy(),
    'captures_minus_TAFT_last': captures_minus_TAFT_last.copy(),
    'all_misses': all_misses.copy(),
    'non_captures_minus_all_misses': non_captures_minus_all_misses.copy(),
    'all_first_misses': all_first_misses.copy(),
}

# normalize schema + dedupe within each dataset
datasets = {k: compare_events.dedupe_within(compare_events.ensure_event_schema(v)) for k, v in datasets_raw.items()}

comparisons = compare_events.build_comparisons([
    {'a': 'captures', 'b': 'no_capture', 'key': 'captures_vs_no_capture'},

    {'a': 'persist_nonfinal', 'b': 'giveup_GUAT_last'},
    {'a': 'persist_middle',  'b': 'giveup_GUAT_last'},

    {'a': 'persist_nonfinal', 'b': 'giveup_GUAT_last_plus_single_miss'},
    {'a': 'persist_middle',   'b': 'giveup_GUAT_last_plus_single_miss'},

    {'a': 'giveup_single_miss', 'b': 'giveup_GUAT_last'},
    {'a': 'giveup_single_miss', 'b': 'persist_both_first'},

    {'a': 'persist_GUAT_nonfinal', 'b': 'persist_TAFT_nonfinal'},

    {'a': 'giveup_GUAT_last', 'b': 'capture_TAFT_last'},

    {'a': 'captures_minus_TAFT_last', 'b': 'capture_TAFT_last'},

    {'a': 'captures', 'b': 'all_misses'},

    {'a': 'non_captures_minus_all_misses', 'b': 'all_misses'},
    {'a': 'non_captures_minus_all_misses', 'b': 'all_first_misses'},
])


compare_events.validate(datasets, comparisons)


# Run

In [ ]:
reload(compare_events)

In [ ]:
cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

runs = compare_events.run_all_comparisons(
    comparisons, datasets, pn.spikes_df, pn.monkey_information, cfg,
    align_by_stop_end=False
)


# SELECT

In [ ]:
# capture vs no capture
event_a_df=valid_captures_df
event_b_df=filtered_no_capture_stops_df

event_a_label = 'valid_captures_df'
event_b_label = 'filtered_no_capture_stops_df'


# persist vs give up
event_a_df=both_nonfinal
event_b_df=giveup_GUAT_last

event_a_label = 'both_nonfinal'
event_b_label = 'giveup_GUAT_last'


# persist (shrunken) vs give up
event_a_df=both_middle
event_b_df=giveup_GUAT_last

event_a_label = 'both_middle'
event_b_label = 'giveup_GUAT_last'


# persist vs give up (expanded)
event_a_df=both_nonfinal
event_b_df=giveup_GUAT_last_plus_single_miss

event_a_label = 'both_nonfinal'
event_b_label = 'giveup_GUAT_last_plus_single_miss'


# persist (shrunken) vs give up (expanded)
event_a_df=both_nonfinal
event_b_df=giveup_GUAT_last_plus_single_miss

event_a_label = 'both_nonfinal'
event_b_label = 'giveup_GUAT_last_plus_single_miss'

# give up in single miss and give up in GUAT
event_a_df=one_stop_miss_df
event_b_df=giveup_GUAT_last

event_a_label = 'one_stop_miss_df'
event_b_label = 'giveup_GUAT_last'


# give up in single miss and not give up in persist_both_first
event_a_df=one_stop_miss_df
event_b_df=persist_both_first

event_a_label = 'one_stop_miss_df'
event_b_label = 'persist_both_first'

# GUAT first several vs TAFT first several
event_a_df=persist_GUAT_nonfinal
event_b_df=persist_TAFT_nonfinal

event_a_label = 'persist_GUAT_nonfinal'
event_b_label = 'persist_TAFT_nonfinal'


# GUAT last vs TAFT last
event_a_df=giveup_GUAT_last
event_b_df=capture_TAFT_last

event_a_label = 'giveup_GUAT_last'
event_b_label = 'capture_TAFT_last'

# capture (not in TAFT) vs TAFT last
valid_captures_df_subset = valid_captures_df[~valid_captures_df['stop_id'].isin(capture_TAFT_last['stop_id'])].copy()

event_a_df=valid_captures_df_subset
event_b_df=capture_TAFT_last

event_a_label = 'valid_captures_df_subset'
event_b_label = 'capture_TAFT_last'


# all captures vs all misses
event_a_df=valid_captures_df
event_b_df=all_misses

event_a_label = 'valid_captures_df'
event_b_label = 'all_misses'


# all misses vs non-capture stops
non_captures_minus_all_misses = filtered_no_capture_stops_df[~filtered_no_capture_stops_df['stop_id'].isin(all_misses['stop_id'])].copy()

event_a_df=non_captures_minus_all_misses
event_b_df=all_misses

event_a_label = 'non_captures_minus_all_misses'
event_b_label = 'all_misses'


# all first misses vs non-capture stops
non_captures_minus_all_misses = filtered_no_capture_stops_df[~filtered_no_capture_stops_df['stop_id'].isin(all_misses['stop_id'])].copy()

all_first_misses = pd.concat([one_stop_miss_df[shared_columns], 
                                         GUAT_first[shared_columns],
                                         TAFT_first[shared_columns]
                                         ])


event_a_df=non_captures_minus_all_misses
event_b_df=all_first_misses

event_a_label = 'non_captures_minus_all_misses'
event_b_label = 'all_first_misses'





# run class

In [ ]:
# capture vs no capture
event_a_df=valid_captures_df
event_b_df=filtered_no_capture_stops_df

event_a_label = 'valid_captures_df'
event_b_label = 'filtered_no_capture_stops_df'

print(f'{event_a_label} vs {event_b_label}')
print('=====================================================')


event_a_df['stop_time'] = event_a_df['stop_id_start_time']
event_b_df['stop_time'] = event_b_df['stop_id_start_time']


cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

an = core_stops_psth.create_psth_around_stops(pn.spikes_df, pn.monkey_information,
                                                 event_a_df=event_a_df,
                                                 event_b_df=event_b_df,
                                                 event_a_label=event_a_label,
                                                 event_b_label=event_b_label,
                                                 config=cfg,
                                                 )

fig2 = an.plot_comparison(cluster_idx=0)
plt.show()


windows = {
    "pre_bump(-0.3–0.0)": (-0.3, 0.0),
    "early_dip(0.0–0.3)": (0.0, 0.3),
    "late_rebound(0.3–0.8)": (0.3, 0.8),
}
    
summary = psth_postprocessing.compare_windows(an, windows, alpha=0.05)
summary_sub = summary.loc[summary['sig_FDR'], ['cluster', 'window', 'p', 'cohens_d', 'sig_FDR']].copy()
summary_sub = summary_sub.sort_values(["sig_FDR", "window", "p"], ascending=[False, True, True])
print(summary_sub)


In [ ]:


windows = {
    "pre_baseline(-0.5–-0.2)": (-0.5, -0.2),
    "pre_bump(-0.2–-0.1)": (-0.2, -0.1),
    "pre_bump2(-0.1–0.0)": (-0.1, 0.0),
    # "pre_bump3(-0.2--0.0)": (-0.2, -0.0),
    "early_dip(0.0–0.1)": (0.0, 0.1),
    "early_rebound(0.1–0.2)": (0.1, 0.2),
    "early_rebound2(0.2–0.3)": (0.2, 0.3),
    "early_rebound3(0.3–0.4)": (0.3, 0.4),
    "early_rebound4(0.4–0.5)": (0.4, 0.5),
    "late_rebound(0.5–0.8)": (0.5, 0.8),
    "post_tail(0.8–1.2)": (0.8, 1.2),
}

summary = psth_postprocessing.compare_windows(an, windows, alpha=0.05)
summary_sub = summary[summary['sig_FDR']].copy()
df = summary_sub.sort_values(["sig_FDR", "window", "p"], ascending=[False, True, True])
df

In [ ]:
summary.sort_values(["sig_FDR", "p"], ascending=[False, True]).head(20)

# Other analysis

## AUC: just pre or post

In [ ]:
import numpy as np
from typing import Iterable, Sequence, Tuple, Optional
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# --- core: build X,y with optional unit selection ---
def build_Xy(an, t0: float, t1: float, units: Optional[Sequence] = None, standardize: bool = False) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    seg = an.psth_data['segments']; time = an.psth_data['psth']['time_axis']; bw = an.config.bin_width
    i0 = int(np.searchsorted(time, t0, side='left')); i1 = int(np.searchsorted(time, t1, side='right'))
    Xa = seg['event_a'][:, i0:i1, :].mean(axis=1) / bw   # nA x U
    Xb = seg['event_b'][:, i0:i1, :].mean(axis=1) / bw   # nB x U
    X = np.vstack([Xa, Xb])                              # (nA+nB) x U
    y = np.r_[np.ones(len(Xa), int), np.zeros(len(Xb), int)]

    # select units if requested
    unit_ids = an.clusters  # original cluster labels
    if units is not None:
        # 'units' can be indices into columns OR original cluster labels
        if all(isinstance(u, (int, np.integer)) for u in units) and set(units).issubset(set(range(X.shape[1]))):
            cols = np.array(list(units), int)
        else:
            # map labels -> column indices
            col_map = {lab: j for j, lab in enumerate(unit_ids)}
            cols = np.array([col_map[u] for u in units], int)
        X = X[:, cols]
        unit_ids = unit_ids[cols]

    if standardize:
        sc = StandardScaler().fit(X)
        X = sc.transform(X)

    return X, y, unit_ids

def decode_auc_cv(X: np.ndarray, y: np.ndarray, k: int = 5, seed: int = 0, C: float = 1.0, class_weight=None) -> Tuple[float, float]:
    cv = StratifiedKFold(k, shuffle=True, random_state=seed)
    aucs = []
    for tr, te in cv.split(X, y):
        clf = LogisticRegression(max_iter=1000, C=C, class_weight=class_weight, solver='lbfgs')
        clf.fit(X[tr], y[tr])
        p = clf.predict_proba(X[te])[:, 1]
        aucs.append(roc_auc_score(y[te], p))
    aucs = np.asarray(aucs, float)
    return float(np.mean(aucs)), float(np.std(aucs))

# --- 1) per-unit (single-cluster) AUCs ---
def per_unit_auc_df(an, window=(-0.3, 0.0), k=5, seed=0, standardize=False, include_sd=True) -> pd.DataFrame:
    X, y, unit_ids = build_Xy(an, *window, standardize=standardize)
    rows = []
    for j, cid in enumerate(unit_ids):
        m, s = decode_auc_cv(X[:, [j]], y, k=k, seed=seed)
        rows.append({'cluster': int(cid) if str(cid).isdigit() else cid, 'auc': float(m), 'sd_cv': float(s)})
    df = pd.DataFrame(rows).sort_values('auc', ascending=False).reset_index(drop=True)
    df['delta'] = df['auc'] - 0.5
    df['rank'] = np.arange(1, len(df) + 1)
    cols = ['rank', 'cluster', 'auc', 'delta'] + (['sd_cv'] if include_sd else [])
    return df[cols]


# --- 2) subset decoding (choose a list of units) ---
def subset_auc(an, units: Sequence, window=(-0.3, 0.0), k=5, seed=0, standardize=True) -> Tuple[float, float]:
    X, y, _ = build_Xy(an, *window, units=units, standardize=standardize)
    return decode_auc_cv(X, y, k=k, seed=seed)

# --- 3) neuron-drop / top-K curve ---
def neuron_drop_curve(an, window=(-0.3, 0.0), k=5, seed=0, rank_by='univariate', standardize=True, max_k=None):
    """
    rank_by: 'univariate' (per-unit AUC vs labels), or 'coef' (coef magnitude from a full model)
    returns dict with 'order' (cluster IDs in rank order) and 'curve' (list of AUCs for K=1..max_k)
    """
    X, y, unit_ids = build_Xy(an, *window, standardize=standardize)

    # ranking
    if rank_by == 'univariate':
        scores = np.full(X.shape[1], -np.inf)
        for j in range(X.shape[1]):
            try:
                scores[j] = roc_auc_score(y, X[:, j])
                scores[j] = max(scores[j], 1 - scores[j])  # make it >= 0.5
            except Exception:
                scores[j] = 0.5
        order = np.argsort(-scores)  # high to low
    elif rank_by == 'coef':
        clf = LogisticRegression(max_iter=1000, C=1.0, solver='lbfgs').fit(X, y)
        scores = np.abs(clf.coef_.ravel())
        order = np.argsort(-scores)
    else:
        raise ValueError("rank_by must be 'univariate' or 'coef'")

    order_ids = unit_ids[order]
    Kmax = X.shape[1] if max_k is None else min(max_k, X.shape[1])
    curve = []
    for K in range(1, Kmax + 1):
        cols = order[:K]
        auc_mean, _ = decode_auc_cv(X[:, cols], y, k=k, seed=seed)
        curve.append(auc_mean)
    return {'order_ids': order_ids, 'curve': np.array(curve), 'rank_scores': scores[order]}

# --- 4) convenience wrappers you can call directly ---
def auc_all_units(an, window=(-0.3, 0.0), **cvkw):
    X, y, _ = build_Xy(an, *window, standardize=True)
    return decode_auc_cv(X, y, **cvkw)

def auc_single_unit(an, unit, window=(-0.3, 0.0), **cvkw):
    return subset_auc(an, units=[unit], window=window, **cvkw)

def auc_topK_units(an, K, window=(-0.3, 0.0), rank_by='univariate', **cvkw):
    rank = neuron_drop_curve(an, window=window, rank_by=rank_by, **{k:v for k,v in cvkw.items() if k in ('k','seed','standardize')})
    return float(rank['curve'][K-1]), rank


import matplotlib.pyplot as plt

def plot_unit_auc_bar(df, top=None):
    dd = df.copy()
    if top is not None:
        dd = dd.head(top)
    fig, ax = plt.subplots(figsize=(8, max(3, 0.35*len(dd))))
    ax.barh(dd['cluster'].astype(str), dd['auc'])
    ax.axvline(0.5, linestyle='--', linewidth=1)
    ax.set_xlabel('AUC')
    ax.set_ylabel('Cluster')
    ax.set_title('Single-unit AUC')
    ax.invert_yaxis()
    plt.tight_layout()
    return fig, ax


In [ ]:
# All clusters

m, s = auc_all_units(an, window=(-0.3, 0.0), k=5, seed=0)
print('pre-stop AUC (all units):', round(m,3))

m, s = auc_all_units(an, window=(0.05, 0.35), k=5, seed=0)
print('post-stop AUC (all units):', round(m,3))

In [ ]:
# Per cluster
df = per_unit_auc_df(an, window=(-0.3, 0.0), k=5, seed=0, standardize=True)
print(df.to_string(index=False, formatters={'auc': '{:.3f}'.format,
                                            'delta': '{:+.3f}'.format,
                                            'sd_cv': '{:.3f}'.format}))

# plot all units
fig, ax = plot_unit_auc_bar(df, top=5)
plt.show()


In [ ]:
# top k curve

res = neuron_drop_curve(an, window=(-0.3, 0.0), k=5, seed=0, rank_by='univariate', standardize=True)
print('best K=5 AUC:', float(res['curve'][4]))
# res['order_ids'] gives the unit order; res['curve'] is AUC for K=1..Kmax


In [ ]:
# pick subset
m, s = subset_auc(an, units=[2, 5], window=(-0.3, 0.0), k=5, seed=0)  # if your labels are 12,27,44
m

## AUC: both pre and post

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# --- reuseable bits ---

def build_Xy(an, t0, t1, standardize=False):
    seg = an.psth_data['segments']; time = an.psth_data['psth']['time_axis']; bw = an.config.bin_width
    i0 = int(np.searchsorted(time, t0, side='left')); i1 = int(np.searchsorted(time, t1, side='right'))
    Xa = seg['event_a'][:, i0:i1, :].mean(axis=1) / bw
    Xb = seg['event_b'][:, i0:i1, :].mean(axis=1) / bw
    X = np.vstack([Xa, Xb])
    y = np.r_[np.ones(len(Xa), int), np.zeros(len(Xb), int)]
    if standardize:
        sc = StandardScaler().fit(X)
        X = sc.transform(X)
    return X, y

def decode_auc_cv(X, y, k=5, seed=0, C=1.0, class_weight=None):
    cv = StratifiedKFold(k, shuffle=True, random_state=seed)
    aucs = []
    for tr, te in cv.split(X, y):
        clf = LogisticRegression(max_iter=1000, C=C, class_weight=class_weight, solver='lbfgs')
        clf.fit(X[tr], y[tr])
        p = clf.predict_proba(X[te])[:, 1]
        aucs.append(roc_auc_score(y[te], p))
    return float(np.mean(aucs)), float(np.std(aucs))

def per_unit_auc_df(an, window=(-0.3, 0.0), k=5, seed=0, standardize=True) -> pd.DataFrame:
    seg = an.psth_data['segments']; time = an.psth_data['psth']['time_axis']; bw = an.config.bin_width
    i0 = int(np.searchsorted(time, window[0], side='left')); i1 = int(np.searchsorted(time, window[1], side='right'))
    Xa = seg['event_a'][:, i0:i1, :].mean(axis=1) / bw
    Xb = seg['event_b'][:, i0:i1, :].mean(axis=1) / bw
    X = np.vstack([Xa, Xb]); y = np.r_[np.ones(len(Xa), int), np.zeros(len(Xb), int)]
    unit_ids = an.clusters

    if standardize:
        sc = StandardScaler().fit(X)
        X = sc.transform(X)

    rows = []
    for j, cid in enumerate(unit_ids):
        m, s = decode_auc_cv(X[:, [j]], y, k=k, seed=seed)
        rows.append({'cluster': int(cid) if str(cid).isdigit() else cid, 'auc': float(m), 'sd_cv': float(s)})
    df = pd.DataFrame(rows).sort_values('auc', ascending=False).reset_index(drop=True)
    df['delta'] = df['auc'] - 0.5
    df['rank'] = np.arange(1, len(df) + 1)
    return df[['rank', 'cluster', 'auc', 'delta', 'sd_cv']]

# --- pre+post in one go ---

def per_unit_auc_pre_post(an,
                          pre=(-0.3, 0.0),
                          post=(0.05, 0.35),
                          k=5, seed=0, standardize=True):
    df_pre  = per_unit_auc_df(an, window=pre,  k=k, seed=seed, standardize=standardize).rename(columns={'auc':'auc_pre','delta':'delta_pre','sd_cv':'sd_pre'})
    df_post = per_unit_auc_df(an, window=post, k=k, seed=seed, standardize=standardize).rename(columns={'auc':'auc_post','delta':'delta_post','sd_cv':'sd_post'})
    wide = pd.merge(df_pre[['cluster','auc_pre','sd_pre']], df_post[['cluster','auc_post','sd_post']],
                    on='cluster', how='inner')
    wide['delta'] = wide['auc_post'] - wide['auc_pre']
    wide = wide.sort_values(['auc_post'], ascending=False).reset_index(drop=True)

    long = pd.concat([
        pd.DataFrame({'cluster': df_pre['cluster'],  'window': 'pre',  'auc': df_pre['auc_pre']  if 'auc_pre' in df_pre else df_pre['auc']}),
        pd.DataFrame({'cluster': df_post['cluster'], 'window': 'post', 'auc': df_post['auc_post'] if 'auc_post' in df_post else df_post['auc']}),
    ], ignore_index=True)
    return {'wide': wide, 'pre': df_pre, 'post': df_post, 'long': long}

# --- plots ---

def plot_unit_auc_bar(df, top=None, title='Single-unit AUC', xlabel='AUC'):
    dd = df.copy()
    if top is not None:
        dd = dd.head(top)
    fig, ax = plt.subplots(figsize=(8, max(3, 0.35*len(dd))))
    ax.barh(dd['cluster'].astype(str), dd['auc'])
    ax.axvline(0.5, linestyle='--', linewidth=1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Cluster')
    ax.set_title(title)
    ax.invert_yaxis()
    plt.tight_layout()
    return fig, ax

def plot_unit_auc_pre_post_bars(res, top=None, title='Single-unit AUC: pre vs post'):
    pre = res['pre'][['cluster','auc_pre']].rename(columns={'auc_pre':'auc'})
    post = res['post'][['cluster','auc_post']].rename(columns={'auc_post':'auc'})
    # common order by max(pre, post)
    order = (pd.merge(pre, post, on='cluster', how='inner')
               .assign(mx=lambda d: d[['auc_x','auc_y']].max(axis=1))
               .sort_values('mx', ascending=False)['cluster'])
    pre = pre.set_index('cluster').loc[order].reset_index()
    post = post.set_index('cluster').loc[order].reset_index()
    if top is not None:
        pre = pre.head(top); post = post.head(top)
    H = max(3, 0.35*len(pre))
    fig, axes = plt.subplots(1, 2, figsize=(14, H), sharey=True)
    for ax, df_, lab in zip(axes, [pre, post], ['pre', 'post']):
        ax.barh(df_['cluster'].astype(str), df_['auc'])
        ax.axvline(0.5, linestyle='--', linewidth=1)
        ax.set_xlabel('AUC')
        ax.set_title(lab)
        ax.invert_yaxis()
    fig.suptitle(title)
    plt.tight_layout(rect=[0,0,1,0.95])
    return fig, axes

def plot_unit_auc_scatter(wide, title='Pre vs Post single-unit AUC'):
    fig, ax = plt.subplots(figsize=(5.5,5.5))
    ax.scatter(wide['auc_pre'], wide['auc_post'], s=20, alpha=0.8)
    ax.plot([0,1],[0,1], linestyle='--', linewidth=1)
    ax.axvline(0.5, linestyle=':', linewidth=1); ax.axhline(0.5, linestyle=':', linewidth=1)
    ax.set_xlabel('AUC (pre)')
    ax.set_ylabel('AUC (post)')
    ax.set_title(title)
    ax.set_xlim(0.3, 1.0); ax.set_ylim(0.3, 1.0)
    ax.grid(True, alpha=0.2)
    plt.tight_layout()
    return fig, ax

def plot_auc_delta_bar(wide, top=None, title='Δ AUC (post − pre) by unit'):
    dd = wide.copy().sort_values('delta', ascending=False)
    if top is not None:
        dd = dd.head(top)
    H = max(3, 0.35*len(dd))
    fig, ax = plt.subplots(figsize=(8, H))
    ax.barh(dd['cluster'].astype(str), dd['delta'])
    ax.axvline(0.0, linestyle='--', linewidth=1)
    ax.set_xlabel('Δ AUC (post − pre)')
    ax.set_ylabel('Cluster')
    ax.set_title(title)
    ax.invert_yaxis()
    plt.tight_layout()
    return fig, ax


In [ ]:
# compute both
res = per_unit_auc_pre_post(an, pre=(-0.30, 0.00), post=(0.05, 0.35), k=5, seed=0, standardize=True)

# bar charts for each window
plot_unit_auc_pre_post_bars(res, top=15)

# pre vs post scatter
plot_unit_auc_scatter(res['wide'])

# delta bar (who improves after the stop?)
plot_auc_delta_bar(res['wide'], top=15)

# if you only want a post-stop bar:
post_only = res['post'].rename(columns={'auc_post':'auc'})
plot_unit_auc_bar(post_only, top=15, title='Single-unit AUC (post-stop)', xlabel='AUC (post)')


## dPCA (demixed PCA): time vs outcome

Note: it’s across all neurons together.

In [ ]:
# assuming you have dpca_time_outcome(an) defined:
rep = dpca_utils.run_dpca_report(an, dpca_fn=dpca_utils.dpca_time_outcome, pre_window=(-0.3, 0.0), post_window=(0.05, 0.35),
                      top_k=20, n_perm=1000, seed=0, show=True)

print('EVR1 obs:', rep['evr_perm']['evr1_obs'], 'p=', rep['evr_perm']['p_value'])
print('R^2 (reconstruction):', rep['reconstruction']['R2'])
print('AUC all units pre/post:', rep['auc_all_units'])

# top outcome neurons
print(rep['top_outcome_units'].head(10).to_string(index=False))


In [ ]:
out = rep['out']

# Global vs within-block normalization (helps intuition):
evr_global = out['outcome']['explained_var']           # what you plotted
evr_within = evr_global / np.sum(evr_global) if np.sum(evr_global)>0 else evr_global
print('Outcome EVR (global):', evr_global[:5])
print('Outcome EVR (within-outcome):', evr_within[:5])  # sums to 1 over outcome comps

# Do the blocks add up? (time + outcome ≤ 1 because interaction/residual isn’t modeled):
print('sum(time EVR) + sum(outcome EVR) =',
      np.sum(out['time']['explained_var']) + np.sum(out['outcome']['explained_var']))

# Is it significant?
print('Outcome EVR1:', rep['evr_perm']['evr1_obs'], '  p=', rep['evr_perm']['p_value'])


## Targeted dim-red (‘outcome axis’)

Idea. Learn a single weight vector w that best predicts event_a vs event_b from pre-stop activity (linear/ridge). Then project full trajectories onto w.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LogisticRegression

def outcome_axis(an, train_window=(-0.3, 0.0), alpha=1.0,
                 model='ridge', standardize=True, return_trials=True):
    """
    Learn a 1D 'outcome axis' on a training window, then project peri-stop trajectories.

    model: 'ridge' (your original) or 'logreg'
    standardize: z-score units using training-window data
    return_trials: also return per-trial projections (for CIs / AUC)
    """
    if not getattr(an, 'psth_data', None):
        an.run_full_analysis()

    seg  = an.psth_data['segments']
    time = an.psth_data['psth']['time_axis']
    bw   = an.config.bin_width
    i0   = int(np.searchsorted(time, train_window[0], side='left'))
    i1   = int(np.searchsorted(time, train_window[1], side='right'))

    # training features (mean rate in window)
    Xa = seg['event_a'][:, i0:i1, :].mean(axis=1) / bw   # nA x U
    Xb = seg['event_b'][:, i0:i1, :].mean(axis=1) / bw   # nB x U
    X  = np.vstack([Xa, Xb])                             # N x U
    y  = np.r_[np.ones(len(Xa), int), np.zeros(len(Xb), int)]

    scaler = None
    if standardize:
        scaler = StandardScaler().fit(X)
        X = scaler.transform(X)

    # fit linear readout
    if model == 'ridge':
        clf = Ridge(alpha=alpha).fit(X, y)
        w = clf.coef_.astype(float)                      # U
        b = float(getattr(clf, 'intercept_', 0.0))
    elif model == 'logreg':
        clf = LogisticRegression(penalty='l2', C=1.0/alpha, solver='lbfgs', max_iter=1000)
        clf.fit(X, y)
        w = clf.coef_.ravel().astype(float)
        b = float(clf.intercept_[0])
    else:
        raise ValueError("model must be 'ridge' or 'logreg'")

    # mean trajectories over time (T x U)
    traj_a = (seg['event_a'] / bw).mean(axis=0)   # T x U
    traj_b = (seg['event_b'] / bw).mean(axis=0)   # T x U
    if standardize:
        # apply the same scaler column-wise to trajectories
        traj = np.vstack([traj_a, traj_b])        # (2T) x U
        traj = scaler.transform(traj)
        traj_a, traj_b = traj[:len(time)], traj[len(time):]

    # project means (time courses)
    proj_a = traj_a @ w + b
    proj_b = traj_b @ w + b

    out = {
        'w': w, 'b': b, 'proj_time': time,
        'proj_a': proj_a, 'proj_b': proj_b,
        'clusters': an.clusters,
        'model': model, 'train_window': train_window,
        'standardize': standardize
    }

    if return_trials:
        # per-trial projections across time (for CIs/AUC without refit)
        A = seg['event_a'] / bw   # nA x T x U
        B = seg['event_b'] / bw   # nB x T x U
        if standardize:
            # flatten trials×time, transform, then reshape back
            AT = A.reshape(-1, A.shape[2])
            BT = B.reshape(-1, B.shape[2])
            AT = scaler.transform(AT); BT = scaler.transform(BT)
            A = AT.reshape(A.shape);  B = BT.reshape(B.shape)
        out['scores_a'] = A @ w + b   # nA x T
        out['scores_b'] = B @ w + b   # nB x T

    return out

def cosine_with_dpca(outcome_axis_w, dpca_out):
    """Cosine similarity between your outcome axis and dPCA outcome comp-1."""
    w1 = outcome_axis_w.ravel()
    w2 = dpca_out['outcome']['components'][0].ravel()
    return float((w1 @ w2) / (np.linalg.norm(w1) * np.linalg.norm(w2) + 1e-12))


In [ ]:
# Learn the axis (train on pre-stop), get time courses

out = outcome_axis(
    an,
    train_window=(-0.3, 0.0),   # train the readout here
    alpha=1.0,                  # ridge strength (or 1/C if model='logreg')
    model='ridge',              # or 'logreg'
    standardize=True,
    return_trials=True
)

# quick plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot(out['proj_time'], out['proj_a'], label=getattr(an, 'event_a_label', 'event_a'))
plt.plot(out['proj_time'], out['proj_b'], label=getattr(an, 'event_b_label', 'event_b'))
plt.axvline(0, ls='--', lw=1)
plt.xlabel('Time (s)'); plt.ylabel('Projection score'); plt.legend(); plt.tight_layout()
plt.show()


In [ ]:
# AUC in any window using the fixed axis (no refit)

import numpy as np
from sklearn.metrics import roc_auc_score

def window_auc_from_scores(out, window):
    t = out['proj_time']
    i0 = int(np.searchsorted(t, window[0], side='left'))
    i1 = int(np.searchsorted(t, window[1], side='right'))
    sa = out['scores_a'][:, i0:i1].mean(axis=1)   # per-trial scores in window
    sb = out['scores_b'][:, i0:i1].mean(axis=1)
    y  = np.r_[np.ones(len(sa), int), np.zeros(len(sb), int)]
    s  = np.r_[sa, sb]
    return roc_auc_score(y, s)

auc_pre  = window_auc_from_scores(out, (-0.3, 0.0))
auc_post = window_auc_from_scores(out, (0.05, 0.35))
print('AUC pre:', round(auc_pre,3), '  AUC post:', round(auc_post,3))


In [ ]:
from sklearn.linear_model import Ridge

def outcome_axis(an, train_window=(-0.3, 0.0), alpha=1.0):
    if not getattr(an, 'psth_data', None):
        an.run_full_analysis()
    seg = an.psth_data['segments']
    time = an.psth_data['psth']['time_axis']
    bw = an.config.bin_width

    i0 = int(np.searchsorted(time, train_window[0], side='left'))
    i1 = int(np.searchsorted(time, train_window[1], side='right'))
    Xa = seg['event_a'][:, i0:i1, :].mean(axis=1) / bw    # nA x U
    Xb = seg['event_b'][:, i0:i1, :].mean(axis=1) / bw    # nB x U
    X = np.vstack([Xa, Xb])
    y = np.r_[np.ones(len(Xa), int), np.zeros(len(Xb), int)]

    model = Ridge(alpha=alpha).fit(X, y)
    w = model.coef_.astype(float)                         # U

    # project mean trajectories over time
    traj_a = (seg['event_a'] / bw).mean(axis=0)          # T x U
    traj_b = (seg['event_b'] / bw).mean(axis=0)          # T x U
    proj_a = traj_a @ w                                   # T
    proj_b = traj_b @ w                                   # T
    return {'w': w, 'proj_time': an.psth_data['psth']['time_axis'],
            'proj_a': proj_a, 'proj_b': proj_b, 'clusters': an.clusters}


In [ ]:
# Optional: bootstrap CI bands on the time courses

import numpy as np

def mean_ci_band(scores, alpha=0.05, iters=1000, seed=0):
    # scores: trials x time
    rng = np.random.default_rng(seed)
    n, T = scores.shape
    boots = np.empty((iters, T))
    for i in range(iters):
        idx = rng.integers(0, n, n)
        boots[i] = scores[idx].mean(0)
    lo, hi = np.percentile(boots, [100*alpha/2, 100*(1-alpha/2)], axis=0)
    return scores.mean(0), lo, hi

ma, lo_a, hi_a = mean_ci_band(out['scores_a'])
mb, lo_b, hi_b = mean_ci_band(out['scores_b'])

t = out['proj_time']
plt.figure(figsize=(8,4))
plt.plot(t, ma, label=getattr(an, 'event_a_label','event_a'))
plt.fill_between(t, lo_a, hi_a, alpha=0.2, linewidth=0)
plt.plot(t, mb, label=getattr(an, 'event_b_label','event_b'))
plt.fill_between(t, lo_b, hi_b, alpha=0.2, linewidth=0)
plt.axvline(0, ls='--', lw=1); plt.xlabel('Time (s)'); plt.ylabel('Projection score'); plt.legend(); plt.tight_layout()
plt.show()


In [ ]:
# Compare to dPCA outcome axis (alignment)
dp = dpca_time_outcome(an)
cos = cosine_with_dpca(out['w'], dp)   # uses the helper I gave earlier
print('cosine(outcome_axis, dPCA outcome comp1) =', round(cos,3))


In [ ]:
# Inspect which neurons drive the axis

import pandas as pd
top = (pd.DataFrame({'cluster': an.clusters, 'w': out['w']})
         .assign(abs_w=lambda d: d['w'].abs())
         .sort_values('abs_w', ascending=False)
         .head(15))
print(top.to_string(index=False))


In [ ]:
# Train on a different window or use logistic

out = outcome_axis(an, train_window=(0.05, 0.35), model='logreg', alpha=1.0)


# quick plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot(out['proj_time'], out['proj_a'], label=getattr(an, 'event_a_label', 'event_a'))
plt.plot(out['proj_time'], out['proj_b'], label=getattr(an, 'event_b_label', 'event_b'))
plt.axvline(0, ls='--', lw=1)
plt.xlabel('Time (s)'); plt.ylabel('Projection score'); plt.legend(); plt.tight_layout()
plt.show()


### ridgeline

In [ ]:
def safe_hedges_g(x, y, eps=1e-3, trim=0.0):
    import numpy as np
    x, y = np.asarray(x, float), np.asarray(y, float)
    if trim > 0:
        qx = np.quantile(x, [trim/2, 1-trim/2]); x = np.clip(x, qx[0], qx[1])
        qy = np.quantile(y, [trim/2, 1-trim/2]); y = np.clip(y, qy[0], qy[1])
    n1, n2 = len(x), len(y)
    if n1 < 2 or n2 < 2: return 0.0
    s1, s2 = np.var(x, ddof=1), np.var(y, ddof=1)
    sp = np.sqrt(((n1-1)*s1 + (n2-1)*s2) / max(n1+n2-2, 1))
    sp = max(sp, eps)
    d = (np.mean(x) - np.mean(y)) / sp
    J = 1 - 3/(4*(n1+n2)-9) if (n1+n2) > 2 else 1.0
    return float(J*d)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

def _kde_safe(x, grid, bw=None):
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    if x.size < 3:
        return np.zeros_like(grid)
    kde = gaussian_kde(x, bw_method=bw)
    return kde(grid)

def plot_ridge_effect_by_window(summary_df, order='effect', bw=None):
    """
    Ridgeline of Cohen's d distributions across windows (aggregated over clusters).
    summary_df: output from summarize_epochs()/compare_windows()
                needs ['window','cohens_d'] (and optionally 'sig_FDR').
    order: 'effect' (windows ordered by |median d|) or 'as_is' (given order).
    """
    df = summary_df.copy()
    df = df[np.isfinite(df['cohens_d'])]
    if df.empty:
        raise ValueError('no finite effect sizes in summary_df')

    # choose window order
    windows = list(df['window'].unique())
    if order == 'effect':
        med = df.groupby('window', as_index=False)['cohens_d'].median()
        windows = med.reindex(med['cohens_d'].abs().sort_values(ascending=False).index)['window'].tolist()

    # common x-range
    lo = np.nanpercentile(df['cohens_d'], 1)
    hi = np.nanpercentile(df['cohens_d'], 99)
    pad = 0.1 * (hi - lo + 1e-9)
    xgrid = np.linspace(lo - pad, hi + pad, 500)

    fig, ax = plt.subplots(figsize=(8, 0.7 * max(3, len(windows))))
    offset = 0.0
    gap = 1.2

    for w in windows:
        x = df.loc[df['window'] == w, 'cohens_d'].values
        y = _kde_safe(x, xgrid, bw=bw)
        if y.max() > 0:
            y = y / y.max()  # normalize height
        ax.fill_between(xgrid, offset, offset + y, alpha=0.6, linewidth=0)
        ax.plot(xgrid, offset + y, linewidth=1.5)
        ax.text(xgrid[0], offset + 0.05, str(w), va='bottom', fontsize=9)
        offset += gap

    ax.axvline(0, linestyle='--', linewidth=1)
    ax.set_yticks([])
    ax.set_xlabel("Cohen's d (event_a − event_b)")
    ax.set_title('Effect ridgeline across windows (distribution over clusters)')
    plt.tight_layout()
    return fig, ax


In [ ]:
summary = psth_postprocessing.summarize_epochs(an)   # or compare_windows(...)
plot_ridge_effect_by_window(summary)
plt.show()


## volcano plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_volcano(summary_df, p_col='p', effect_col='cohens_d', by_window=True, sig_col='sig_FDR'):
    """
    Volcano: effect size vs -log10(p).
    summary_df: from summarize_epochs()/compare_windows()/sliding_window_stats
    p_col: 'p' or 'p_value' (whatever your table uses)
    by_window: if True, one panel per window; else single panel with colors by window
    """
    df = summary_df.copy()
    if p_col not in df.columns:
        # try common names
        p_col = 'p_value' if 'p_value' in df.columns else p_col
    df = df[np.isfinite(df[effect_col]) & np.isfinite(df[p_col])]
    if df.empty:
        raise ValueError('no finite rows to plot')

    df['_mlog10p'] = -np.log10(df[p_col].clip(lower=1e-300))

    if by_window:
        windows = list(df['window'].unique())
        n = len(windows)
        fig, axes = plt.subplots(n, 1, figsize=(7, max(3, 2.2*n)), sharex=True)
        if n == 1:
            axes = [axes]
        for ax, w in zip(axes, windows):
            sub = df[df['window'] == w]
            if sig_col in sub.columns:
                sig = sub[sig_col].astype(bool).values
            else:
                sig = np.zeros(len(sub), bool)
            ax.scatter(sub[effect_col][~sig], sub['_mlog10p'][~sig], s=14, alpha=0.6)
            ax.scatter(sub[effect_col][ sig], sub['_mlog10p'][ sig], s=20, alpha=0.9)
            ax.axvline(0, linestyle='--', linewidth=1)
            ax.set_ylabel('-log10(p)')
            ax.set_title(str(w))
        axes[-1].set_xlabel("Effect size (e.g., Cohen's d)")
        plt.tight_layout()
        return fig, axes
    else:
        fig, ax = plt.subplots(figsize=(7,5))
        for w in df['window'].unique():
            sub = df[df['window'] == w]
            ax.scatter(sub[effect_col], sub['_mlog10p'], s=14, alpha=0.7, label=str(w))
        ax.axvline(0, linestyle='--', linewidth=1)
        ax.set_xlabel("Effect size (e.g., Cohen's d)")
        ax.set_ylabel('-log10(p)')
        ax.legend()
        plt.tight_layout()
        return fig, ax


In [ ]:
plot_volcano(summary)
plt.show()

## Noise correlations (pre-stop)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import mannwhitneyu

def noise_correlations(an, window=(-0.3, 0.0)):
    if not getattr(an, 'psth_data', None):
        an.run_full_analysis()
    seg = an.psth_data['segments']; time = an.psth_data['psth']['time_axis']
    bw = an.config.bin_width
    i0 = int(np.searchsorted(time, window[0], side='left'))
    i1 = int(np.searchsorted(time, window[1], side='right'))

    def corr_offdiag(arr):  # arr: trials x time x units
        if arr.shape[0] < an.config.min_trials:
            return np.array([]), None
        X = arr[:, i0:i1, :].mean(axis=1) / bw            # trials x units
        X = X - X.mean(axis=0, keepdims=True)             # residuals
        C = np.corrcoef(X, rowvar=False)                  # units x units
        iu = np.triu_indices_from(C, k=1)
        return C[iu], C

    rA, CA = corr_offdiag(seg['event_a'])
    rB, CB = corr_offdiag(seg['event_b'])

    stat, p = (np.nan, np.nan)
    if len(rA) > 0 and len(rB) > 0:
        stat, p = mannwhitneyu(rA, rB, alternative='two-sided')

    return {
        'offdiag_event_a': rA, 'offdiag_event_b': rB,
        'matrix_event_a': CA, 'matrix_event_b': CB,
        'mw_stat': stat, 'mw_p': p
    }


# ---------- utilities ----------
def _iqr(x):
    x = np.asarray(x, float)
    q1, q3 = np.nanpercentile(x, [25, 75])
    return float(q3 - q1)

def _sym_lims(*arrays, q=99.0, min_lim=0.2):
    a = np.concatenate([np.ravel(np.abs(z[np.isfinite(z)])) for z in arrays if z is not None and np.size(z)])
    if a.size == 0:
        return (-1, 1)
    m = np.nanpercentile(a, q)
    m = max(m, min_lim)
    return (-m, m)

# ---------- 1) distributions + MWU ----------
def plot_noise_corr_distributions(res, title='Noise correlations (off-diagonal)', bins=40):
    rA, rB = res['offdiag_event_a'], res['offdiag_event_b']
    if rA.size == 0 or rB.size == 0:
        print('Not enough trials for at least one condition.')
        return None, None

    lims = _sym_lims(rA, rB, q=99.0, min_lim=0.2)
    edges = np.linspace(lims[0], lims[1], bins+1)

    mA, mB = np.nanmedian(rA), np.nanmedian(rB)
    iA, iB = _iqr(rA), _iqr(rB)
    p  = res.get('mw_p', np.nan)

    fig, ax = plt.subplots(figsize=(7, 3.2))
    ax.hist(rA, bins=edges, alpha=0.45, density=True, label='event_a', edgecolor='none')
    ax.hist(rB, bins=edges, alpha=0.45, density=True, label='event_b', edgecolor='none')
    ax.axvline(0, ls='--', c='k', lw=1)
    ax.set_xlim(lims); ax.set_xlabel('Noise correlation r'); ax.set_ylabel('Density')
    ax.set_title(title)
    ax.legend(title=f'mWU p={p:.3g}\nmedian/IQR  a={mA:.3f}/{iA:.3f},  b={mB:.3f}/{iB:.3f}')
    plt.tight_layout()
    return fig, ax

# ---------- 2) matrices (upper triangles) ----------
def plot_noise_corr_matrices(res, clusters=None, vmax_q=99.0, title='Noise correlation matrices'):
    CA, CB = res['matrix_event_a'], res['matrix_event_b']
    if CA is None or CB is None:
        print('Correlation matrices not available.')
        return None, None, None

    M = CA.shape[0]
    mask = np.tril_indices(M, k=0)
    A = CA.copy(); B = CB.copy()
    A[mask] = np.nan; B[mask] = np.nan

    vmin, vmax = _sym_lims(A, B, q=vmax_q)
    cmap = plt.get_cmap('coolwarm').copy(); cmap.set_bad('white')

    fig, axes = plt.subplots(1, 3, figsize=(12, 3.4), constrained_layout=True)
    im0 = axes[0].imshow(A, vmin=vmin, vmax=vmax, cmap=cmap)
    axes[0].set_title('event_a')
    im1 = axes[1].imshow(B, vmin=vmin, vmax=vmax, cmap=cmap)
    axes[1].set_title('event_b')
    D = CB - CA
    D[mask] = np.nan
    dvmin, dvmax = _sym_lims(D, q=vmax_q)
    im2 = axes[2].imshow(D, vmin=dvmin, vmax=dvmax, cmap=cmap)
    axes[2].set_title('event_b − event_a')

    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])
    cbar0 = fig.colorbar(im0, ax=axes[:2], fraction=0.046, pad=0.04)
    cbar0.set_label('r')
    cbar1 = fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
    cbar1.set_label('Δr')

    fig.suptitle(title)
    return fig, axes, (vmin, vmax, dvmin, dvmax)

# ---------- 3) paired scatter (per unit-pair) ----------
def plot_noise_corr_scatter(res, title='Pairwise noise corr: event_a vs event_b'):
    rA, rB = res['offdiag_event_a'], res['offdiag_event_b']
    if rA.size == 0 or rB.size == 0 or rA.size != rB.size:
        print('Off-diagonal vectors missing or mismatched sizes.')
        return None, None

    fig, ax = plt.subplots(figsize=(4.2, 4.2))
    ax.scatter(rA, rB, s=10, alpha=0.5)
    lims = _sym_lims(rA, rB, q=99.0)
    ax.plot(lims, lims, ls='--', c='k', lw=1)
    ax.set_xlim(lims); ax.set_ylim(lims)
    ax.set_xlabel('r (event_a)'); ax.set_ylabel('r (event_b)')
    ax.set_title(title)
    # simple summaries
    delta = np.nanmean(rB - rA)
    ax.text(0.05, 0.95, f'Δ mean = {delta:.3f}', transform=ax.transAxes, va='top')
    ax.grid(True, alpha=0.2)
    plt.tight_layout()
    return fig, ax

# ---------- 4) one-call convenience ----------
def noise_corr_report(an, window=(-0.3, 0.0), show=True):
    res = noise_correlations(an, window=window)
    title = f'Noise correlations  window=({window[0]:.2f},{window[1]:.2f})'
    fig1, _ = plot_noise_corr_distributions(res, title=title)
    fig2, _, _ = plot_noise_corr_matrices(res, title=title)
    fig3, _ = plot_noise_corr_scatter(res, title=title)
    if show:
        import matplotlib.pyplot as plt
        plt.show()
    return {'res': res, 'fig_dist': fig1, 'fig_mats': fig2, 'fig_scatter': fig3}


In [ ]:
# # compute + visualize (pre-stop)
out_pre = noise_corr_report(an, window=(-0.30, 0.00))

# post-stop too
out_post = noise_corr_report(an, window=(0.05, 0.35))

# If you only want a specific view:

# res = noise_correlations(an, window=(0.05, 0.35))
res = noise_correlations(an, window=(-0.30, 0.00))
plot_noise_corr_distributions(res)
plot_noise_corr_matrices(res)
plot_noise_corr_scatter(res)


## Poisson GLM with elastic-net (regularized)

In [ ]:
look it up again
https://chatgpt.com/c/68b22a5f-d034-8331-8a9d-7f36b25a802b

# More plots

## Quickly plot PSTHs for the top significant neurons

In [ ]:
import numpy as np

def plot_top_psths(analyzer, summary: pd.DataFrame, epoch: str, top_k=6):
    # pick significant clusters in the epoch, ranked by |d|
    g = summary[(summary["window"] == epoch) & (summary["sig_FDR"])].copy()
    if g.empty:
        print(f"No significant clusters for {epoch}."); return
    g = g.sort_values("cohens_d", key=lambda s: s.abs(), ascending=False).head(top_k)

    # map string cluster ids back to analyzer cluster indices
    plotted = 0
    for cl_str in g["cluster"]:
        # analyzer.clusters holds original IDs (numeric or str)
        # coerce both sides to string for robust matching
        matches = np.where(np.array(list(map(str, analyzer.clusters))) == str(cl_str))[0]
        if len(matches) == 0: 
            continue
        ci = int(matches[0])
        analyzer.plot_comparison(cluster_idx=ci)  # your existing method
        plotted += 1
    if plotted == 0:
        print("Nothing plotted (no matches).")

# usage
plot_top_psths(an, summary, "early_dip(0.0–0.3)", top_k=7)


## Heatmap of effect sizes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# usage
psth_postprocessing.plot_sig_heatmap(summary)


## Bar chart

In [ ]:
# keep only rows where sig_FDR is True
sig_rows = summary[summary["sig_FDR"]]

# plot effect sizes by epoch
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
sns.barplot(data=sig_rows, x="window", y="cohens_d", hue="cluster", dodge=True)
plt.axhline(0, color="k", lw=1)
plt.ylabel("Cohen's d (capture − miss)")
# change angle of xticks
plt.xticks(rotation=30)
plt.title("Significant neurons across epochs")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


## Bar chart of significant effects per epoch (one bar per cluster)

In [ ]:
def plot_sig_bars(summary: pd.DataFrame, epoch: str):
    g = summary[(summary["window"] == epoch) & (summary["sig_FDR"])].copy()
    if g.empty:
        print(f"No significant clusters for {epoch}."); return
    g = g.sort_values("cohens_d", key=lambda s: s.abs(), ascending=False)

    fig, ax = plt.subplots(figsize=(10, max(3, 0.35 * len(g))))
    ax.barh(g["cluster"], g["cohens_d"])
    ax.axvline(0, color="k", lw=1, alpha=0.5)
    ax.set_xlabel("Cohen's d (capture − miss)")
    ax.set_ylabel("Cluster")
    ax.set_title(f"Significant clusters in {epoch}")
    plt.tight_layout(); plt.show()

# usage
plot_sig_bars(summary, "pre_bump(-0.3–0.0)")
plot_sig_bars(summary, "early_dip(0.0–0.3)")
plot_sig_bars(summary, "late_rebound(0.3–0.8)")


# Appendix

## Other anchors for stops
https://docs.google.com/document/d/1nJOQlqsyq7eOVtUlL8TAAacC25vXeQvlGfDI-SbQrKE/edit?tab=t.0

In [ ]:
pn.ff_dataframe.columns

In [ ]:
import numpy as np
import pandas as pd

def _finite_diff(y, t):
    dy = np.gradient(y, t)  # robust to uneven dt
    return dy

def anchor_peak_decel(df, pre_window_s=0.5):
    # peak negative acceleration in a window ending at stop onset
    # requires per-stop onset; derive from first time of each stop_id
    onset = df.groupby("stop_id")["time"].min()
    t = df["time"].to_numpy()
    a = _finite_diff(df["speed"].to_numpy(), t)  # acceleration
    df2 = df.assign(accel=a)
    rows = []
    for sid, t0 in onset.items():
        m = (df2["stop_id"]==sid) & (df2["time"].between(t0-pre_window_s, t0, inclusive="left"))
        if not m.any(): 
            continue
        i = df2.loc[m, "accel"].idxmin()
        rows.append((sid, df2.at[i, "time"]))
    out = pd.DataFrame(rows, columns=["stop_id","anchor_time"])
    out["anchor_kind"] = "peak_decel"
    return out





## other processing/windows

In [ ]:

# # Per-cluster plots with bands
# fig1 = an.plot_psth(cluster_idx=None, show_individual=False)


# df = psth_postprocessing.export_psth_to_df(an)              # all clusters
# df_c0 = psth_postprocessing.export_psth_to_df(an, [0])      # just the first cluster


# windows = {
#     "pre_bump(-0.3–0.0)": (-0.3, 0.0),
#     "early_dip(0.0–0.3)": (0.0, 0.3),
#     "late_rebound(0.3–0.8)": (0.3, 0.8),
# }


# windows = {
#     "pre_baseline(-0.6–-0.3)": (-0.6, -0.3),
#     "pre_bump(-0.3–0.0)": (-0.3, 0.0),
#     "early_dip(0.0–0.1)": (0.0, 0.1),
#     "early_rebound(0.1–0.3)": (0.1, 0.3),
#     "late_rebound(0.3–0.8)": (0.3, 0.8),
#     "post_tail(0.8–1.5)": (0.8, 1.5),
# }

## Validate near-miss single stops

In [ ]:
# Check whether "near-miss" stops (one_stop_w_ff_df) are truly not part of a stop cluster.  
# A stop cluster is defined as ≥ 2 stops where each consecutive stop is within 50 cm (cumulative distance).

# --- Step 1: Create one-stop dataframe and assign cluster IDs
pn.make_one_stop_w_ff_df()
pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information)

# --- Step 2: Build stop-cluster summary
stop_cluster_df = (
    pn.monkey_information.loc[pn.monkey_information['whether_new_distinct_stop'], ['point_index', 'stop_cluster_id']]
    .copy()
)
stop_cluster_df['num_stops_in_cluster'] = (
    stop_cluster_df.groupby('stop_cluster_id')['point_index'].transform('count')
)

# --- Step 3: Merge cluster info into one-stop dataframe (if not already present)
if 'stop_cluster_id' not in pn.one_stop_w_ff_df.columns:
    pn.one_stop_w_ff_df = pn.one_stop_w_ff_df.merge(
        stop_cluster_df.rename(columns={'point_index': 'first_stop_point_index'}),
        on='first_stop_point_index',
        how='left'
    )

# --- Step 4: Inspect any one-stop rows that actually fall in a multi-stop cluster
pn.one_stop_w_ff_df[pn.one_stop_w_ff_df['num_stops_in_cluster'] > 1]


## Check dt between stops in clusters

In [ ]:
import pandas as pd


# Expand so each stop_index gets its own row
GUAT_expanded = pn.GUAT_trials_df.explode("stop_indices").reset_index(drop=True)

# Optionally rename column
GUAT_expanded = GUAT_expanded.rename(columns={"stop_indices": "stop_point_index"})
GUAT_expanded['stop_time'] = pn.monkey_information['time'].loc[GUAT_expanded['stop_point_index']].values

TAFT_expanded = pn.TAFT_trials_df.explode("stop_indices").reset_index(drop=True)

# Optionally rename column
TAFT_expanded = TAFT_expanded.rename(columns={"stop_indices": "stop_point_index"})
TAFT_expanded['stop_time'] = pn.monkey_information['time'].loc[TAFT_expanded['stop_point_index']].values


# group TAFT_expanded by stop_cluster_id and drop the last row of each group
TAFT_expanded.sort_values('stop_point_index', inplace=True)
persist_TAFT_nonfinal = TAFT_expanded.groupby('stop_cluster_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [ ]:
GUAT_expanded['dt'] = GUAT_expanded['stop_time'].diff()
TAFT_expanded['dt'] = TAFT_expanded['stop_time'].diff()
GUAT_sub = GUAT_expanded[GUAT_expanded['dt'] < 0.5]
TAFT_sub = TAFT_expanded[TAFT_expanded['dt'] < 0.5]
GUAT_sub


In [ ]:
sns.histplot(GUAT_sub[['dt']])
plt.show()

In [ ]:
sns.histplot(TAFT_sub[['dt']])
plt.show()

In [ ]:
GUAT_sub[['dt']].describe()

In [ ]:
TAFT_sub[['dt']].describe()

## check inter-stop intervals

In [ ]:
reload(process_monkey_information)

In [ ]:
pn.monkey_information = process_monkey_information.add_whether_new_distinct_stop_and_stop_id(pn.monkey_information)

In [ ]:
pn.monkey_information

In [ ]:

unique_stops_df = (
    pn.monkey_information[['stop_id', 'point_index', 'time', 'stop_id_start_time', 'stop_id_end_time', 'stop_id_duration']].groupby("stop_id", as_index=False, sort=False)
    .first()
    .reset_index(drop=True)
)
unique_stops_df

In [ ]:
unique_stops_df = get_stops_utils.extract_unique_stops(pn.monkey_information)
unique_stops_df

In [ ]:
unique_stops_df = get_stops_utils.extract_unique_stops(pn.monkey_information)
onsets = unique_stops_df['time'].to_numpy()

In [ ]:
returned = get_stops_utils.plot_inter_stop_intervals(onsets)
ax1 = returned['ax1']

# add an additional vertical line to the linear plot
additional_vline = 0.3
ax1.axvline(additional_vline, linestyle="--", color="b", alpha=0.8, label=f"x = {additional_vline}s")
ax1.legend()
plt.show()

## check stop durations

In [ ]:
plt.figure()
ax = sns.histplot(unique_stops_df['stop_id_duration'], bins=100)
ax.set_xlim(0, 1)
plt.show()

## add stop_id

In [ ]:
pn.monkey_information = process_monkey_information.add_whether_new_distinct_stop_and_stop_id(pn.monkey_information)
pn.monkey_information

## stop clusters

In [ ]:
pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information, max_cluster_distance=50, use_ff_caught_time_new_to_separate_clusters=False,
                                                                        col_exists_ok=False)

In [ ]:
for max_cluster_distance in [50, 45, 40, 35, 30, 20, 50]:
    print(f'max_cluster_distance: {max_cluster_distance}')
    pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information, max_cluster_distance=max_cluster_distance, use_ff_caught_time_new_to_separate_clusters=False,
                                                                        col_exists_ok=False)

    stop_sub = pn.monkey_information[pn.monkey_information['stop_id'].notna()].copy()
    stop_sub = stop_sub[['stop_id', 'stop_cluster_id']].drop_duplicates()
    # Compute cluster sizes properly
    stop_sub['cluster_size'] = stop_sub.groupby('stop_cluster_id')['stop_id'].transform('size')

    # Show how many rows belong to clusters of size > 1
    print('number of clusters: ', stop_sub['stop_cluster_id'].unique().shape[0])
    print('number of stops: ', stop_sub['stop_id'].unique().shape[0])
    print('number of clusters with more than 1 stop: ', stop_sub[stop_sub['cluster_size'] > 1].shape[0])
    print('--------------------------------')


In [ ]:
num_stops = pn.monkey_information['stop_id'].unique().shape[0]
num_clusters = pn.monkey_information['stop_cluster_id'].unique().shape[0]
num_clusters

## censor_mask_for_event

censor_mdef censor_mask_for_event(t0, all_stops, time_axis, pad=0.15):
    abs_times = t0 + time_axis
    dmin = np.min(np.abs(abs_times[:,None] - all_stops[None,:]), axis=1)
    keep = (dmin >= pad) | np.isclose(abs_times, t0)
    return keep  # bool, shape (len(time_axis),)
ask_for_event